# Hypobeta: analysis of hereditary variants

The objective of this project is to test different command line tools and define the best approach for analysis of hereditary variants.

We have 4 exomes pertaining to a family of four, healthy parents (DHB-14.01 and DHB-14.02) and affected kids (DHB-14.00 and DHB-18.00).

Start date: 2017/12/02  
End date: 2018/01/20

### Workspace
cmc@hpc3_vm2s  
cmc@mochi  

Working_directory:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis  
  
BED_directory:../bed_truseq/no_chr_truseq-dna-exome-targeted-regions-manifest-v1-2.bed  
fastq_files:../fastq_files/  
reference_genome:/nfs/qnapugdg8tb3a/nfs_references/human/1kg/hs37d5/hs37d5.fa  
snap_reference_genome:../nfs/qnapugdg8tb3a/nfs_references/human/1kg/hs37d5/hs37d5.fa

# Analysis

### Preliminar fastqc analysis

\# Archivo .fof  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ find ../fastq_files/ -name "DHB*.fastq.gz" > fastq_files.fof  
  
\# Comando paralelizado de control de calidad con fastqc  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --joblog ./fastq_QC/raw_QC/fastqc_parallel.log -j14 "fastqc -o ./fastq_QC/raw_QC {}" :::: fastq_files.fof  


### Trimming with seqtk and cutadapt

\# Comando paralelizado de trimming con seqtk  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --er {gz} 'echo {gz}' :::: fastq_files.fof | parallel --joblog ./fastq_trimmed/seqtk_trimming/trimming_fastq_gz.log -j14 "seqtk trimfq {}.gz | gzip > ./fastq_trimmed/seqtk_trimming/{/.}-seqtk-trimmed.fastq.gz"  

\# Comando paralelizado de trimming con cutadapt  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --er {gz} 'echo {gz}' :::: fastq_files.fof | parallel --joblog ./fastq_trimmed/cutadapt_trimming/cutadapt_trimming_fastq_gz.log -j14 "cutadapt -q 30 {}.gz | gzip > ./fastq_trimmed/cutadapt_trimming/{/.}-cutadapt-trimmed.fastq.gz"  


\# Comando paralelizado de control de calidad con fastqc  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --joblog ./fastq_QC/seqtk_trimmed_QC/fastqc_seqtk_parallel.log -j14 "fastqc -o ./fastq_QC/seqtk_trimmed_QC {}" :::: fastq_seqtk_trimmed.fof


### Merge fastq files

In [ ]:
#! usr/bin/python3.6

# 2017-12-31, Carolina Monzo
# merge_fastq.py

import os
import re   
import pandas as pd

def fastq_dataframe(fastq_files):
    '''
    Function to take lines from fof file and create a sorted pandas dataframe with info from fastqs
    Input: list of fastq files read from the fof file
    Output: sorted dataframe with the fastq name metadata
    '''

    # Regular expression to create an annonymous dictionary
    dict_list = []
    rex = re.compile(r"\./fastq_trimmed/(seqtk|cutadapt)_trimming/(?P<sample>DHB-\d+\.\d+)_L(?P<lane>\d+)_R(?P<read>\d+).+")
    for fastq in fastq_files:
        m = rex.match(fastq)
        dicc = m.groupdict()
        dicc['fastq_file'] = fastq.split('/')[-1]
        dicc['fastq_path'] = str(fastq)
        dict_list.append(dicc)

    # Read fastq info array into a pandas dataframe
    df_fastq = pd.DataFrame(dict_list, columns = ['sample', 'lane', 'read', 'fastq_file', 'fastq_path'])
    # Sort by sample, lane and read
    df_fastq_sorted = df_fastq.sort_values(by=["fastq_path"], ascending=[True]).reset_index(drop=True)

    return(df_fastq_sorted)

def zcat_fastq(df_fastq_sorted):
    '''
    Function to generate the zcat commands, it selects R1 and R2 from sorted samples
    Input: dataframe of fastq sorted names
    Output: cmd file with the zcat commands of ordered fastqs to merge
    '''

    # Create list by sample
    samples = list(df_fastq_sorted['sample'].unique())

    # Get trimming tool
    
    trimming_tool = list(df_fastq_sorted['fastq_file'])[0].split('-')[2]
    
    # Set zcat template

    for i in range(len(samples)):
            
            sample_fastqs = list(df_fastq_sorted[df_fastq_sorted['sample']==samples[i]]['fastq_path'])
            R1_list = sample_fastqs[0::2]
            R2_list = sample_fastqs[1::2]
            merged_fastq_name = df_fastq_sorted['sample'].unique()[i]
            R1_str = "zcat {} | gzip > ./fastq_merged/fastq_merged_{}/{}_R1_merged.fastq.gz".format(' '.join(R1_list), \
                trimming_tool, merged_fastq_name)
            R2_str = "zcat {} | gzip > ./fastq_merged/fastq_merged_{}/{}_R2_merged.fastq.gz".format(' '.join(R2_list), \
                trimming_tool, merged_fastq_name)
                                    
            # Write the cmd.sh file
            with open('cmd_zcat_{}_fastq.sh'.format(trimming_tool), 'a') as cmd_file:
                cmd_file.write(R1_str + '\n')
                cmd_file.write(R2_str + '\n')

def main():
    '''
    Function to sort the orders
    '''
    fastq_fof = 'fastq_seqtk_trimmed.fof'

    with open(fastq_fof) as fof:
            fastq_files = fof.read().splitlines()


    df_fastq_sorted = fastq_dataframe(fastq_files)

    zcat_fastq(df_fastq_sorted)



if __name__ == '__main__':
    main()


\# Ejecución del script inicial  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ python3.6 merge_fastq.py  
  
\# Ejemplo de comando estándar de mergeo de archivos .fastq   
zcat ./fastq_trimmed/seqtk_trimming/DHB-14.00_L004_R1_seqtk-trimmed.fastq.gz ./fastq_trimmed/seqtk_trimming/DHB-14.00_L005_R1_seqtk-trimmed.fastq.gz | gzip > ./fastq_merged/DHB-14.00_R1_merged.fastq  
  
\# Ejecución en paralelo del script generado ‘cmd_zcat_fastq.sh’  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --joblog ./fastq_merged/merge_fastq.log -j15 :::: cmd_zcat_fastq.sh  


### Quality control with fastqc

\# Ejecución en paralelo del control de calidad con fastqc  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --joblog ./fastq_QC/seqtk_merged_QC/fastqc_seqtk_merged_parallel.log -j16 "fastqc -o ./fastq_QC/seqtk_merged_QC {}" :::: merged_fastq_seqtk.fof 


### Mapping with snap and bwa

\# Indexado del genoma de referencia para mapeo con BWA  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ samtools faidx /nfs/qnapugdg8tb3a/nfs_references/human/1kg/hs37d5  
  
\# Indexado del genoma de referencia para mapeo con SNAP  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/snap_reference_genome$ /home/cmc/software/snap/snap-aligner index hs37d5.fa . -exact -sm


In [ ]:
#! usr/bin/python3.6

# 2018-01-03, Carolina Monzo
# map.py

import os
import pandas as pd
import re


def merged_fastq_dataframe(fastq_files):
    '''
    Function to take lines from fof file to create a sorted pandas dataframe with info from merged fastqs
    Input: list of fastq files read from the fof file
    Ouptut: sorted dataframe with the fastq name metadata
    '''
    #<TODO> join this function with the fastq_dataframe function from merge_fastq.py
        #Ideally we would only create the rex variable outside and pass it as input to the function
        #Then we create an if statement and chose the corresponding columns for the dataframe
    


    # Regular expression to create an annonymous dictionary
    dict_list = []

    rex = re.compile(r"\./fastq_merged/fastq_merged_(seqtk|cutadapt)/(?P<sample>DHB-\d+\.\d+)_R(?P<read>\d+).+")

    for fastq in fastq_files:
        m = rex.match(fastq)
        dicc = m.groupdict()
        dicc['fastq_file'] = fastq.split('/')[-1]
        dicc['fastq_path'] = str(fastq)
        dict_list.append(dicc)

    # Read fastq info array into a pandas dataframe
    
    df_fastq = pd.DataFrame(dict_list, columns = ['sample', 'read', 'fastq_file', 'fastq_path'])

    # Sort by sample and read

    df_fastq_sorted = df_fastq.sort_values(by=['fastq_path'], ascending=[True]).reset_index(drop=True)

    return(df_fastq_sorted)

def map_fastq(df_fastq_sorted):
    '''
    Function to create commands to map merged fastq files, we have
        a R1 and R2 fastq corresponding to each sample
    Input: dataframe with metadata from merged fastq files
    Output: files with commands for mapping with BWA and SNAP
    '''
    # Create list by sample

    samples = list(df_fastq_sorted['sample'].unique())

    # Get trimming tool

    trimming_tool = list(df_fastq_sorted['fastq_path'])[0].split('/')[2].split('_')[-1]

    # Set templates

    for i in range(len(samples)):
        sample_fastqs = list(df_fastq_sorted[df_fastq_sorted['sample']==samples[i]]['fastq_path'])
        R1_list = sample_fastqs[0]
        R2_list = sample_fastqs[1]
        output_bam_name = df_fastq_sorted['sample'].unique()[i]

        # Create template for bwa
        
        cmd_bwa = "time bwa mem -M -L 5 -t 5 /storage/ethernus_miscellanea/References/human/1kg/hs37d5/hs37d5.fa.gz \
            {} {} 2> ./mapping_bwa_mem/bwa_{}/{}_bwa_mem.err | samtools sort -O bam -o \
            ./mapping_bwa_mem/bwa_{}/{}.{}.sorted.bam; time samtools index ./mapping_bwa_mem/bwa_{}/{}.{}.sorted.bam \
            ".format(R1_list, R2_list, trimming_tool, output_bam_name, trimming_tool, output_bam_name, trimming_tool, 
            trimming_tool, output_bam_name, trimming_tool)

        # Write the cmd.sh file

        with open('cmd_bwa_mem_{}.sh'.format(trimming_tool), 'a') as cmd_file:
            cmd_file.write(cmd_bwa + '\n')

        # Create template for SNAP

        cmd_snap = "time /home/cmc/software/snap/snap-aligner paired /storage/ethernus_miscellanea/\
            scratch_local/workspace/cmc_projects_tmp/hipobeta_work/snap_reference_genome/ {} 
            {} -o -bam ./mapping_snap/snap_{}/{}.{}.unsorted.bam".format(R1_list, R2_list, trimming_tool, 
            output_bam_name, trimming_tool)
        # Create template for samtools sort and remove unsorted bam file

        cmd_samtools_sort = "time samtools sort ./mapping_snap/snap_{}/{}.{}.unsorted.bam -O bam -o \
            ./mapping_snap/snap_{}/{}.{}.sorted.bam; rm -f ./mapping_snap/snap_{}/{}.{}.unsorted.bam; \
            time samtools index ./mapping_snap/snap_{}/{}.{}.sorted.bam".format(trimming_tool, output_bam_name, 
            trimming_tool, trimming_tool, output_bam_name, trimming_tool, trimming_tool, output_bam_name, 
            trimming_tool, trimming_tool, output_bam_name, trimming_tool)

        # Write the cmd.sh file

        with open('cmd_snap_{}.sh'.format(trimming_tool), 'a') as cmd_file:
            cmd_file.write(cmd_snap + '; ' + cmd_samtools_sort + '\n')

def main():
    '''
    Function to sort the order of functions to use
    '''

    fastq_fof = 'merged_fastq_seqtk.fof'

    with open(fastq_fof) as fof:
        fastq_files = fof.read().splitlines()

    # Create fastq files dataframe

    df_fastq_sorted = merged_fastq_dataframe(fastq_files)

    # Create BWA, SNAP and STAR commands for mapping

    map_fastq(df_fastq_sorted)



if __name__ == '__main__':
    main()

\# Ejecución del script de gestión  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ python3.6 map.py  
  
\# Ejemplo de comando estándar de mapeo utilizando BWA   
bwa mem -M -L 5 -t 5 /storage/ethernus_miscellanea/References/human/1kg/hs37d5/hs37d5.fa.gz                 ./fastq_merged/fastq_merged_cutadapt/DHB-14.00_R1_merged.fastq.gz ./fastq_merged/fastq_merged_cutadapt/DHB-14.00_R2_merged.fastq.gz 2> ./mapping_bwa_mem/bwa_cutadapt/DHB-14.00_bwa_mem.err | samtools sort -O bam -o                 ./mapping_bwa_mem/bwa_cutadapt/DHB-14.00.cutadapt.sorted.bam; time samtools index ./mapping_bwa_mem/bwa_cutadapt/DHB-14.00.cutadapt.sorted.bam  
  
\# Ejemplo de comando estándar de mapeo utilizando SNAP  
time /home/cmc/software/snap/snap-aligner paired ../snap_reference_genome/ ./fastq_merged/fastq_merged_seqtk/DHB-14.00_R1_merged.fastq.gz ./fastq_merged/fastq_merged_seqtk/DHB-14.00_R2_merged.fastq.gz -o -bam ./mapping_snap/snap_seqtk/DHB-14.00.seqtk.unsorted.bam; time samtools sort ./mapping_snap/snap_seqtk/DHB-14.00.seqtk.unsorted.bam -O bam -o ./mapping_snap/snap_seqtk/DHB-14.00.seqtk.sorted.bam; rm -f ./mapping_snap/snap_seqtk/DHB-14.00.seqtk.unsorted.bam; time samtools index ./mapping_snap/snap_seqtk/DHB-14.00.seqtk.sorted.bam  
  
\# Ejecución en paralelo de los scripts generados  
\# ‘cmd_(bwa_mem|snap)_(cutadapt|seqtk).sh’  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --joblog ./mapping_bwa_mem/bwa_seqtk/bwa_seqtk-20180103.log -j 4 :::: cmd_bwa_mem_seqtk.sh ; 
parallel --joblog ./mapping_snap/snap_cutadapt/snap_cutadapt-20180103.log -j 1 :::: cmd_snap_cutadapt.sh


### Testing trimming and mapping mistakes

\# Comprobación de existencia de líneas blancas en archivos trimmeados con cutadapt  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ zcat ./fastq_merged/fastq_merged_cutadapt/DHB-14.00_R1_merged.fastq.gz | grep -cvE '[^[:space:]]'  
134240  
  
\# Comprobación de existencia de líneas blancas en archivos trimmeados con seqtk  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ zcat ./fastq_merged/fastq_merged_seqtk/DHB-14.00_R1_merged.fastq.gz | grep -cvE '[^[:space:]]'  
0  


### Mapping QC

\# Comando paralelizado para ejecutar samtools flagstat  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --er {bam} 'echo {bam}' :::: bam_snap_seqtk.fof | parallel --joblog ./mapping_QC/snap_seqtk_QC/flagstat-20180105.log -j16 "samtools flagstat {}.bam > ./mapping_QC/snap_seqtk_QC/{/.}_flagstat.txt"  

\# Comando para generar un script en bash para calcular estadísticas de mapeo con picard  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis\$ for file in \$(cat bam_snap_seqtk.fof | cut -d "/" -f4); do echo "picard CollectMultipleMetrics R=/storage/ethernus_miscellanea/References/human/1kg/hs37d5/hs37d5.fa.gz I=./mapping_snap/snap_seqtk/\${file} O=./mapping_QC/snap_seqtk_QC/\$\{file\%.bam} PROGRAM=CollectInsertSizeMetrics PROGRAM=QualityScoreDistribution PROGRAM=CollectGCBiasMetrics PROGRAM=MeanQualityByCycle 2> ./mapping_QC/snap_seqtk_QC/${file%.bam}" >> cmd_picard_metrics_snap_seqtk.sh ; done  
  
\# Ejecución en paralelo del control de calidad utilizando picard  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --joblog ./mapping_QC/bwa_seqtk_QC/alignmentsummarymetrics-20180107.log -j16 :::: cmd_picard_alignment_bwa_seqtk.sh  


### Merge bam files

\# Añadir campo @RG a todas las muestras  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis/mapping_bwa_mem/bwa_seqtk$ picard AddOrReplaceReadGroups I=DHB-14.00.seqtk.sorted.bam O=DHB-14.00.seqtk.sorted_rg.bam RGLB=Truseq RGPL=illumina RGSM=DHB-14.00 RGCN=INCLIVA RGPU=AGATGT RGID=DHB-14.00  
  
\# Mergear en un solo archivo .bam  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis/mapping_bwa_mem/bwa_seqtk$ samtools merge quartet_merged.bam DHB-14.00.seqtk.sorted_rg.bam DHB-14.01.seqtk.sorted_rg.bam DHB-14.02.seqtk.sorted_rg.bam DHB-18.00.seqtk.sorted_rg.bam  


### Getting coverage metrics

\# Obtener histograma de coberturas con bedtools  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --er {bam} 'echo {bam}' :::: bam_bwa_seqtk.fof | parallel --joblog ./coverage/bwa_seqtk/bedtools-hist-20180109.log -j10 "bedtools coverage -hist -a {}.bam -b ../bed_truseq/no_chr_truseq-dna-exome-targeted-regions-manifest-v1-2.bed > ./coverage/bwa_seqtk/{/.}-hist.txt"  

\# Obtener cobertura por base con samtools depth  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ parallel --er {bam} 'echo {bam}' :::: bam_bwa_seqtk.fof | parallel --joblog ./coverage/bwa_seqtk/samtools-B-20180110.log -j10 "samtools depth -b ../bed_truseq/no_chr_truseq-dna-exome-targeted-regions-manifest-v1-2.bed {}.bam > /media/qnapugdg8tb_2b/cmc_projects_tmp/hypobeta_work/analysis/coverage/bwa_seqtk/{/.}-B-depth.cpb" 


\# Cálculo de cobertura media por muestra  
cmc@mochi:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis/coverage/bwa_seqtk\$ for file in $(ls \*seqtk-B-depth.cpb); do echo \$file && cat \$file | awk '{x+=\$3}END{print x/NR}'; done


\# Get percentaje of dp10, dp20..  

cmc@mochi:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis/coverage/bwa_seqtk
\$ for file in \$(ls \*seqtk-B-depth.cpb); do echo \$file && cat \$file | awk '{ if (\$3>10) {dp10+=\$3}} {if (\$3>20) {dp20+=\$3}} {if (\$3>30) {dp30+=\$3}} {if (\$3>40) {dp40+=\$3}} {if 
(\$3>50) {dp50+=\$3}} {if (\$3>60) {dp60+=\$3}} {if (\$3>70) {dp70+=\$3}} {if (\$3>80) {dp80+=\$3}} {if (\$3>90) {dp90+=\$3}} END{print dp10/NR; print dp20/NR; print dp30/NR; print dp40
/NR; print dp50/NR; print dp60/NR; print dp70/NR; print dp80/NR; print dp90/NR}'; done                                                                                         
DHB-14.00.seqtk-B-depth.cpb


### Variant calling with freebayes and samtools mpileup

\# Llamado de variantes con freebayes  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ freebayes -f /nfs/qnapugdg8tb3a/nfs_references/human/1kg/hs37d5/hs37d5.fa --min-alternate-fraction 0.05 --report-all-haplotype-alleles --min-base-quality 10 --pooled-continuous -C 4 ./mapping_bwa_mem/bwa_seqtk/quartet_merged.bam > ./variant_calling/bwa_seqtk_freebayes/quartet_merged_freebayes.vcfs

\# Llamado de variantes con samtools mpileup  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ samtools mpileup -v -d 200 -C50 -u -f /nfs/qnapugdg8tb3a/nfs_references/human/1kg/hs37d5/hs37d5.fa ./mapping_bwa_mem/bwa_seqtk/quartet_merged.bam > ./variant_calling/bwa_seqtk_mpileup/quartet_merged_mpileup.vcf 


### Filtering variants with vcftools
  
**Our variants of interest had depths of 12, we decided not to filter by depth in this step and filter by depth directly in the gemini commands**

\# Comando para filtrar variantes
cmc@mochi:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis/variant_calling/bwa_seqtk_freebayes
$ vcftools --gzvcf quartet_merged_freebayes.vcf --minDP 20 --minGQ 20 --recode --recode-INFO-all --out quartet_merged_freebayes-dp20q20


### Decompose and normalize files to separate by parsimonious alleles

\# Comando para descomponer y normalizar archivos .vcf  
cmc@mochi:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis/variant_calling/bwa_seqtk_freebayes
$ vt decompose -s quartet_merged_freebayes-dp20q20.recode.vcf | vt normalize -r /nfs/qnapugdg8tb3a/nfs_references/human/1kg/hs37d5/hs37d5.fa -n - > quartet_merged_freebayes-dp20q20.recode-norm.vcf


### Variant annotation with snpEff and VEP

\# Comando para anotar variantes utilizando snpEff  
cmc@mochi:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis/variant_calling/bwa_seqtk_freebayes
$ java -jar /software/snpeff/snpEff-v4.3k/snpEff.jar GRCh37.75 -formatEff -classic quartet_merged_freebayes-dp20q20.recode-norm.vcf > quartet_merged_freebayes-dp20q20.recode-norm-snpEff.vcf  

\# Comando para anotar variantes utilizando VEP
cmc@mochi:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis/variant_calling/bwa_seqtk_freebayes
$ vep -i quartet_merged_freebayes-dp20q20.recode-norm.vcf --format vcf --cache --dir /home/ugdg_admin/.vep/ --assembly GRCh37 --offline --force_overwrite --sift b -- polyphen b --symbol --no_stats --af_1kg --af_1kb --af_esp --af_gnomad--vcf -o quartet_merged_freebayes-dp20q20.recode-norm-VEP.vcf


### Loading and filtering using gemini

\# Carga de archivos .vcf multisample a gemini  
cmc@vlinux16ugdg3:/nfs/production2d/cmc_projects_tmp/hypobeta_work/analysis$ gemini load -v ./variant_calling/bwa_seqtk_freebayes/quartet_merged_freebayes-dp20q20.recode-norm-VEP.vcf -t VEP -p ./heritage/hypobeta_quartet.ped --cores 16 ~/workspace/hypobeta_work/hypobeta_quartet-VEP.db  

\# Comando final de filtrado para herencia heterocigota compuesta  
cmc@vlinux16ugdg3:~/workspace/hypobeta_work$ gemini comp_hets hypobeta_quartet-VEP.db --columns "chrom, start, end, gene, impact, sift_score, polyphen_score, ref, alt, aaf_1kg_eur, cadd_scaled, in_exac, aaf_gnomad_all, max_aaf_all" --filter "impact_severity != 'LOW' and aaf_1kg_eur < 0.1 and cadd_scaled > 20"  

\# Comando final de filtrado para herencia autosómica recesiva  
cmc@vlinux16ugdg3:~/workspace/hypobeta_work$ gemini autosomal_recessive hypobeta_quartet-VEP.db --columns "chrom, start, end, gene, impact, sift_score, polyphen_score, ref, alt, aaf_1kg_eur, cadd_scaled, in_exac, aaf_gnomad_all, max_aaf_all" --filter "impact_severity != 'LOW' and aaf_1kg_eur < 0.1 and cadd_scaled > 20"


## Software
FastQC v0.11.5  
Seqtk v1.2-r94  
Cutadapt v1.14  
Bwa v0.7.16a-r1181  
SNAP v1.0beta.23  
samtools v1.5  
python v3.6  
picard v2.12.1  
bedtools v2.26.0  
freebayes v1.1.0  
vcftools v0.1.15  
vt v0.5772-60f436c3  
snpEff v4.3k  
VEP versions: ensembl (90.4a44397), ensembl-funcgen (90.e775c00), ensembl-io (90.9a148ea), ensembl-variation (90.58bf949), ensembl-vep (90.5)